# Ilab 算法竞赛

In [ ]:
print("Hello ilab !!!")

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms

import torch.optim as optim
import torchvision.models as models

import PIL.Image as Image

print('start download')

start download


In [2]:
!wget best_m.onnx https://hzk-1.oss-cn-hangzhou.aliyuncs.com/best_m.onnx

--2022-06-11 12:37:48--  http://best_m.onnx/
Resolving best_m.onnx (best_m.onnx)... failed: Name or service not known.
wget: unable to resolve host address ‘best_m.onnx’
--2022-06-11 12:37:49--  https://hzk-1.oss-cn-hangzhou.aliyuncs.com/best_m.onnx
Resolving hzk-1.oss-cn-hangzhou.aliyuncs.com (hzk-1.oss-cn-hangzhou.aliyuncs.com)... 118.31.219.217
Connecting to hzk-1.oss-cn-hangzhou.aliyuncs.com (hzk-1.oss-cn-hangzhou.aliyuncs.com)|118.31.219.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83880434 (80M) [application/octet-stream]
Saving to: ‘best_m.onnx’

best_m.onnx         100%[===================>]  79.99M  4.48MB/s    in 22s     

2022-06-11 12:38:11 (3.57 MB/s) - ‘best_m.onnx’ saved [83880434/83880434]

FINISHED --2022-06-11 12:38:11--
Total wall clock time: 23s
Downloaded: 1 files, 80M in 22s (3.57 MB/s)


In [3]:
!ls /home/jovyan

best_m.onnx  work


In [1]:
pip install --upgrade opencv-python

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 60.9 MB 6.7 MB/s eta 0:00:01    |████                            | 7.8 MB 6.4 MB/s eta 0:00:09     |████▍                           | 8.4 MB 6.4 MB/s eta 0:00:09     |██████                          | 11.2 MB 6.4 MB/s eta 0:00:08     |████████████                    | 22.9 MB 5.7 MB/s eta 0:00:07     |████████████▌                   | 23.8 MB 5.7 MB/s eta 0:00:07     |██████████████▋                 | 27.8 MB 6.4 MB/s eta 0:00:06     |███████████████▍                | 29.3 MB 6.4 MB/s eta 0:00:05     |████████████████                | 30.6 MB 5.3 MB/s eta 0:00:06     |█████████████████▌              | 33.3 MB 5.3 MB/s eta 0:00:06     |██████████████████▉             | 35.8 MB 5.3 MB/s eta 0:00:05     |████████████████████████████████| 60.7 MB 6.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade opencv-python-headless

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 48.3 MB 285 kB/s eta 0:00:011   |█▉                              | 2.7 MB 5.8 MB/s eta 0:00:088 MB 458 kB/s eta 0:01:31     |█████████████▌                  | 20.3 MB 1.6 MB/s eta 0:00:18     |████████████████████████████▍   | 42.8 MB 12.0 MB/s eta 0:00:01
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.4.0.46
    Uninstalling opencv-python-headless-4.4.0.46:
      Successfully uninstalled opencv-python-headless-4.4.0.46
Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import numpy as np
import random

print('cv2版本：',cv2.__version__)

cv2版本： 4.6.0


In [4]:
import cv2
net = cv2.dnn.readNetFromONNX("/home/jovyan/best_m.onnx")
print('load successful')

load successful


In [8]:
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt


print('cv2版本：',cv2.__version__)

def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), scaleup=False, stride=32):
   
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    # 只进行下采样 因为上采样会让图片模糊
    # (for better test mAP) scale_up = False 对于大于new_shape（r<1）的原图进行缩放,小于new_shape（r>1）的不变
    if not scaleup:  # only scale down, do not scale up (for better test mAP)
        r = min(r, 1.0)
    ratio = r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    # 这里的取余操作可以保证padding后的图片是32的整数倍(416x416)，如果是(512x512)可以保证是64的整数倍
    # dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    # 在较小边的两侧进行pad, 而不是在一侧pad
    # divide padding into 2 sides
    dw /= 2
    dh /= 2
    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img,ratio,(dw,dh)

def iou(b1,b2):
    b1_x1, b1_y1, b1_x2, b1_y2 = b1[0], b1[1], b1[2], b1[3]
    b2_x1, b2_y1, b2_x2, b2_y2 = b2[:,0], b2[:,1], b2[:,2], b2[:,3]

    inter_rect_x1 = np.maximum(b1_x1, b2_x1)
    inter_rect_y1 = np.maximum(b1_y1, b2_y1)
    inter_rect_x2 = np.minimum(b1_x2, b2_x2)
    inter_rect_y2 = np.minimum(b1_y2, b2_y2)

    inter_area = np.maximum(inter_rect_x2 - inter_rect_x1, 0) * \
                 np.maximum(inter_rect_y2 - inter_rect_y1, 0)

    area_b1 = (b1_x2 - b1_x1) * (b1_y2 - b1_y1)
    area_b2 = (b2_x2 - b2_x1) * (b2_y2 - b2_y1)

    iou = inter_area / np.maximum((area_b1+area_b2-inter_area),1e-6)
    return iou


def non_max_suppression(boxes, conf_thres, nms_thres, ratio=1, pad=(20,20)):
    # 取出batch_size
    bs = np.shape(boxes)[0]
    # xywh___ to____ xyxy
    shape_boxes = np.zeros_like(boxes[:,:,:4])
    shape_boxes[:, :, 0] = boxes[:, :, 0] - boxes[:, :, 2] / 2
    shape_boxes[:, :, 1] = boxes[:, :, 1] - boxes[:, :, 3] / 2
    shape_boxes[:, :, 2] = boxes[:, :, 0] + boxes[:, :, 2] / 2
    shape_boxes[:, :, 3] = boxes[:, :, 1] + boxes[:, :, 3] / 2
    boxes[:, :, :4] = shape_boxes
    boxes[:, :, 5:] *= boxes[:, :, 4:5]

    # output存放每一张图片的预测结果，推理阶段一般是一张图片
    output = []
    for i in range(bs):
        predictions = boxes[i]  # 预测位置xyxy  shape==(12700,85)
        score = np.max(predictions[:, 5:], axis=-1)
        # score = predictions[:,4]  # 存在物体置信度,shape==12700
        mask = score > conf_thres  # 物体置信度阈值mask==[False,False,True......],shape==12700,True将会被保留，False列将会被删除
        detections = predictions[mask]  # 第一次筛选  shape==(115,85)
        class_conf = np.expand_dims(np.max(detections[:,5:],axis=-1),axis=-1)  # 获取每个预测框预测的类别置信度
        class_pred = np.expand_dims(np.argmax(detections[:,5:],axis=-1),axis=-1)  # 获取每个预测框的类别下标
        # 结果堆叠，(num_boxes,位置信息4+包含物体概率1+类别置信度1+类别序号1)
        detections = np.concatenate([detections[:,:4],class_conf,class_pred],axis=-1)  # shape=(numbox,7)

        unique_class = np.unique(detections[:,-1])  # 取出包含的所有类别
        if len(unique_class)==0:
            continue
        best_box = []
        for c in unique_class:
            # 取出类别为c的预测结果
            cls_mask = detections[:,-1] == c
            detection = detections[cls_mask] # shape=(82,7)

            # 包含物体类别概率从高至低排列
            scores = detection[:,4]
            arg_sort = np.argsort(scores)[::-1]  # 返回的是索引
            detection = detection[arg_sort]

            while len(detection) != 0:
                best_box.append(detection[0])
                if len(detection) == 1:
                    break
                # 计算当前置信度最大的框和其它预测框的iou
                ious = iou(best_box[-1],detection[1:])
                detection = detection[1:][ious < nms_thres]  # 小于nms_thres将被保留，每一轮至少减少一个
        output.append(best_box)

    boxes_loc = []
    conf_loc = []
    class_loc = []
    if len(output):
        for i in range(len(output)):
            pred = output[i]
            for i, det in enumerate(pred):
                if len(det):
                    # 将框坐标调整回原始图像中
                    det[0] = (det[0] - pad[0]) / ratio
                    det[2] = (det[2] - pad[0]) / ratio
                    det[1] = (det[1] - pad[1]) / ratio
                    det[3] = (det[3] - pad[1]) / ratio
                    boxes_loc.append([det[0],det[1],det[2],det[3]])
                    conf_loc.append(det[4])
                    class_loc.append(det[5])
    return boxes_loc,conf_loc,class_loc
'''
def plot_one_box(img,boxes,conf,clas_id,line_thickness=3,names=None):
    # 画位置框
    # tl = 框的线宽  要么等于line_thickness要么根据原图im长宽信息自适应生成一个
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1
    color = [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(boxes[0]), int(boxes[1])), (int(boxes[2]),int(boxes[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)

    # 画类别信息框
    label = f'{names[int(clas_id)]} {conf:.2f}'
    print(names[int(clas_id)],c1[0],c1[1],c2[0],c2[1])
    print(conf)
    tf = max(tl - 1, 1)  # label字体的线宽 font thickness
    t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
    c2 = (c1[0] + t_size[0], c1[1] - t_size[1] - 3)
    cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)
    cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

if __name__ == '__main__':
    # # COCO 80类别数据集
    # names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
    #         'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    #         'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
    #         'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
    #         'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    #         'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
    #         'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    #         'cell phone','microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
    #         'teddy bear', 'hair drier', 'toothbrush']
    names = ['stand','sit','fall']
    conf_thres = 0.03
    nms_thres = 0.4
    img_path = '/ilab/datasets/local/Fall-data/test/image-121-4.jpeg'

    frame = cv2.imread(img_path)
    frame = frame[:, :, ::-1]
    img, ratio, (dw,dh) = letterbox(frame)

    # np.ascontiguousarray()将一个内存不连续存储的数组转换为内存连续存储的数组，使得运行速度更快
    blob = cv2.dnn.blobFromImage(np.ascontiguousarray(img), 1/255.0, (img.shape[0], img.shape[1]), swapRB=True, crop=False)
    #net = cv2.dnn.readNetFromONNX("/home/jovyan/best.onnx.1")
    net.setInput(blob)
    
    #net.setInput(blob)
    outs = net.forward()  
    boxes_loc,conf_loc,class_loc = non_max_suppression(outs, conf_thres=conf_thres, nms_thres=nms_thres,ratio=ratio, pad=(dw,dh))

    # 可视化
    for i in range(len(boxes_loc)):
        boxes = boxes_loc[i]
        conf = conf_loc[i]
        clas_id = class_loc[i]
        plot_one_box(frame, boxes, conf, clas_id, line_thickness=3, names=names)
    
    plt.imshow(frame)
    plt.show()
'''
'''
if __name__ == '__main__':
    names = ['stand','sit','fall']
    conf_thres = 0.03
    nms_thres = 0.4
    img_path = '/ilab/datasets/local/Fall-data-large/test/'
    sub = []
    import os
    file_dir = '/ilab/datasets/local/Fall-data-large/test'  #你的文件路径
    def getFlist(path):
        for root, dirs, files in os.walk(file_dir):
            print('files:', files)     #文件名称，返回list类型
        return files
    file_name = getFlist(file_dir)
    count = 0
    for i in range(0,len(file_name)):
        name = img_path + file_name[i]
        frame = cv2.imread(name)
        img, ratio, (dw,dh) = letterbox(frame)

        # np.ascontiguousarray()将一个内存不连续存储的数组转换为内存连续存储的数组，使得运行速度更快
        blob = cv2.dnn.blobFromImage(np.ascontiguousarray(img), 1/255.0, (img.shape[0], img.shape[1]), swapRB=True, crop=False)

        net.setInput(blob)
        outs = net.forward()  # API version>=2022.1
        boxes_loc,conf_loc,class_loc = non_max_suppression(outs, conf_thres=conf_thres, nms_thres=nms_thres,ratio=ratio, pad=(dw,dh))

        # 可视化
        if len(boxes_loc) == 0:
            sub.append(i+' '+'stand'+'  '+str(int(frame.shape[1]/5*2))+' '+str(int(frame.shape[0]/5*2))+' '+str(int(frame.shape[1]/5*3))+' '+str(int(frame.shape[0]/5*3)))
        else:
            maxf=conf_loc[0]
            maxindex=0
            for j in range(1,len(boxes_loc)):
                if conf_loc[j] > maxf:
                    maxf = conf_loc[j]
                    maxindex = j
                
            boxes = boxes_loc[maxindex]
            clas_id = class_loc[maxindex]
            sub.append(i+' '+names[int(clas_id)]+'  '+str(int(boxes[0]))+' '+str(int(boxes[1]))+' '+str(int(boxes[2]))+' '+str(int(boxes[3])))
        count+=1
        if count%10==0:print(count)
'''

cv2版本： 4.6.0


"\nif __name__ == '__main__':\n    names = ['stand','sit','fall']\n    conf_thres = 0.03\n    nms_thres = 0.4\n    img_path = '/ilab/datasets/local/Fall-data-large/test/'\n    sub = []\n    import os\n    file_dir = '/ilab/datasets/local/Fall-data-large/test'  #你的文件路径\n    def getFlist(path):\n        for root, dirs, files in os.walk(file_dir):\n            print('files:', files)     #文件名称，返回list类型\n        return files\n    file_name = getFlist(file_dir)\n    count = 0\n    for i in range(0,len(file_name)):\n        name = img_path + file_name[i]\n        frame = cv2.imread(name)\n        img, ratio, (dw,dh) = letterbox(frame)\n\n        # np.ascontiguousarray()将一个内存不连续存储的数组转换为内存连续存储的数组，使得运行速度更快\n        blob = cv2.dnn.blobFromImage(np.ascontiguousarray(img), 1/255.0, (img.shape[0], img.shape[1]), swapRB=True, crop=False)\n\n        net.setInput(blob)\n        outs = net.forward()  # API version>=2022.1\n        boxes_loc,conf_loc,class_loc = non_max_suppression(outs, conf_thres=conf_

In [9]:
names = ['stand','sit','fall']
conf_thres = 0.2
nms_thres = 0.45
img_path = '/ilab/datasets/local/Fall-data-large/test/'
sub = []
count = 0
nonono=0

In [10]:

import os
file_dir = '/ilab/datasets/local/Fall-data-large/test'  #你的文件路径
def getFlist(path):
    for root, dirs, files in os.walk(file_dir):
        print('files:', files)     #文件名称，返回list类型
    return files
file_name = getFlist(file_dir)

for i in range(0,len(file_name)):
    name = img_path + file_name[i]
    frame = cv2.imread(name)
    img, ratio, (dw,dh) = letterbox(frame)

    # np.ascontiguousarray()将一个内存不连续存储的数组转换为内存连续存储的数组，使得运行速度更快
    blob = cv2.dnn.blobFromImage(np.ascontiguousarray(img), 1/255.0, (img.shape[0], img.shape[1]), swapRB=True, crop=False)

    net.setInput(blob)
    outs = net.forward()  # API version>=2022.1
    boxes_loc,conf_loc,class_loc = non_max_suppression(outs, conf_thres=conf_thres, nms_thres=nms_thres,ratio=ratio, pad=(dw,dh))

    # 可视化
    if len(boxes_loc) == 0:
        sub.append(file_name[i]+' '+'stand'+'  '+str(int(frame.shape[1]/5*2))+' '+str(int(frame.shape[0]/5*2))+' '+str(int(frame.shape[1]/5*3))+' '+str(int(frame.shape[0]/5*3)))
        nonono+=1
    else:
        maxf=conf_loc[0]
        maxindex=0
        for j in range(1,len(boxes_loc)):
            if conf_loc[j] > maxf:
                maxf = conf_loc[j]
                maxindex = j
                
        boxes = boxes_loc[maxindex]
        clas_id = class_loc[maxindex]
        sub.append(file_name[i]+' '+names[int(clas_id)]+'  '+str(int(boxes[0]))+' '+str(int(boxes[1]))+' '+str(int(boxes[2]))+' '+str(int(boxes[3])))
    count+=1
    if count%10==0:print(count)

print(nonono)

files: ['fall-01-cam0-rgb-011.png', 'fall-01-cam0-rgb-001.png', 'fall-01-cam0-rgb-015.png', 'fall-01-cam0-rgb-039.png', 'fall-01-cam0-rgb-070.png', 'fall-01-cam0-rgb-074.png', 'fall-01-cam0-rgb-076.png', 'fall-01-cam0-rgb-087.png', 'fall-01-cam0-rgb-088.png', 'fall-01-cam0-rgb-093.png', 'fall-01-cam0-rgb-114.png', 'fall-01-cam0-rgb-115.png', 'fall-01-cam0-rgb-116.png', 'fall-01-cam0-rgb-126.png', 'fall-01-cam0-rgb-128.png', 'fall-01-cam0-rgb-132.png', 'fall-01-cam0-rgb-158.png', 'fall-01-cam1-rgb-096.png', 'fall-01-cam1-rgb-103.png', 'fall-01-cam1-rgb-108.png', 'fall-01-cam1-rgb-121.png', 'fall-01-cam1-rgb-123.png', 'fall-01-cam1-rgb-134.png', 'fall-01-cam1-rgb-139.png', 'fall-01-cam1-rgb-147.png', 'fall-01-cam1-rgb-157.png', 'fall-02-cam0-rgb-003.png', 'fall-02-cam0-rgb-012.png', 'fall-02-cam0-rgb-014.png', 'fall-02-cam0-rgb-018.png', 'fall-02-cam0-rgb-021.png', 'fall-02-cam0-rgb-022.png', 'fall-02-cam0-rgb-023.png', 'fall-02-cam0-rgb-024.png', 'fall-02-cam0-rgb-033.png', 'fall-02-cam

In [11]:
len(sub)

1445

In [23]:
with open('/home/ilab/submission','w')as f:
    for i in range(len(sub)):
        f.write(sub[i]+'\n')

In [24]:
!cat /home/ilab/submission

fall-01-cam0-rgb-011.png stand  367 98 456 324
fall-01-cam0-rgb-001.png stand  380 97 464 322
fall-01-cam0-rgb-015.png stand  363 97 455 325
fall-01-cam0-rgb-039.png stand  381 93 458 330
fall-01-cam0-rgb-070.png stand  338 77 435 359
fall-01-cam0-rgb-074.png stand  332 76 429 358
fall-01-cam0-rgb-076.png stand  324 75 425 360
fall-01-cam0-rgb-087.png stand  266 81 396 366
fall-01-cam0-rgb-088.png stand  258 82 392 367
fall-01-cam0-rgb-093.png stand  234 104 377 392
fall-01-cam0-rgb-114.png fall  131 340 371 479
fall-01-cam0-rgb-115.png fall  129 330 374 478
fall-01-cam0-rgb-116.png fall  128 318 376 478
fall-01-cam0-rgb-126.png fall  182 333 416 479
fall-01-cam0-rgb-128.png fall  197 359 421 478
fall-01-cam0-rgb-132.png fall  229 366 421 478
fall-01-cam0-rgb-158.png fall  223 375 403 479
fall-01-cam1-rgb-096.png stand  209 0 400 163
fall-01-cam1-rgb-103.png fall  201 14 383 295
fall-01-cam1-rgb-108.png fall  181 15 364 330
fall-01-cam1-rgb-121.png fall  211 70 362 406
fall-01-cam1-rgb

In [ ]:
!ls /ilab/datasets/local/Fall-data-large/test

In [ ]:
'''
import cv2
path='/ilab/datasets/local/Fall-data-large/test/image-2-0-304.jpeg'
spect= cv2.imread(path) # BGR
spect= spect[:, :, ::-1] # RGB
import matplotlib.pyplot as plt
plt.imshow(spect)
plt.axis('off')
plt.show()
'''